In [1]:
%matplotlib inline
import utils ; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [2]:
model_path = "../data/imdb/models/"

## Setting up of Data 

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

2318336/2343108 [============================>.] - ETA: 0s

In [11]:
#sort the dict and get the first ten values
idx_arr = sorted(idx,key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [12]:
idx2word = {v:k for k,v in idx.iteritems()}

In [13]:
#download the reviews using code copied from keras.datasets
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

65544192/65552540 [============================>.] - ETA: 0s

In [18]:
len(x_train)

25000

In [21]:
#print the first sentence
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [23]:
#the first sentence with label
' '.join([idx2word[i] for i in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [24]:
labels_train[:20]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [25]:
## Reduce vocab_size 
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [26]:
lens = np.array(map(len,trn))
(lens.max(), lens.min(),lens.mean())

(2493, 10, 237.71364)

In [27]:
#pad with 0 
seq_len = 500
trn = sequence.pad_sequences(trn, maxlen = seq_len, value=0 )
test = sequence.pad_sequences(test, maxlen = seq_len, value=0 )

In [29]:
trn.shape

(25000, 500)

## Modeling.

single layer NN

In [30]:
#note - 32 is the feature mapping, hence 5000 * 32 = 160000 parameters 
model = Sequential([
        Embedding(vocab_size, 32, input_length = seq_len), 
        Flatten(),
        Dense(100,activation='relu'),
        Dropout(0.7),
        Dense(1,activation='sigmoid')
    ])

In [31]:
model.compile(loss = 'binary_crossentropy',optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 16000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           1600100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [33]:
model.fit(trn,labels_train, validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 13s - loss: 0.4789 - acc: 0.7358 - val_loss: 0.2935 - val_acc: 0.8761
Epoch 2/2
25000/25000 [==============================] - 14s - loss: 0.2067 - acc: 0.9237 - val_loss: 0.3039 - val_acc: 0.8734


## Conv with max pooling 

taking adv of ordered data

In [39]:
conv1 = Sequential([
        Embedding(vocab_size, 32, input_length= seq_len, dropout=0.2),
        Dropout(0.2),
        Convolution1D(64,5,border_mode='same',activation='relu'),
        Dropout(0.2),
        MaxPooling1D(),
        Flatten(),
        Dense(100,activation='relu'),
        Dropout(0.7),
        Dense(1,activation='sigmoid')
    ])

In [40]:
conv1.compile(loss = 'binary_crossentropy', optimizer= Adam(), metrics=['accuracy'])

In [41]:
conv1.fit(trn,labels_train, validation_data=(test,labels_test),nb_epoch=4,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 170s - loss: 0.5148 - acc: 0.7161 - val_loss: 0.3005 - val_acc: 0.8787
Epoch 2/4
25000/25000 [==============================] - 174s - loss: 0.2982 - acc: 0.8778 - val_loss: 0.2944 - val_acc: 0.8740
Epoch 3/4
25000/25000 [==============================] - 186s - loss: 0.2598 - acc: 0.8969 - val_loss: 0.2638 - val_acc: 0.8920
Epoch 4/4
25000/25000 [==============================] - 171s - loss: 0.2410 - acc: 0.9051 - val_loss: 0.2659 - val_acc: 0.8888


In [44]:
conv1.save_weights(model_path + 'conv1.h5')

In [45]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors 

Replicate previous CNN, but with pre-trained word embeddings

In [56]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [57]:
vecs,words,wordidx = load_vectors("../data/glove/results/6B.50d")

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [58]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [59]:
emb = create_emb()

In [61]:
#pass embedding matrix to the keras embedding, and set to non trainable
model = Sequential([
        Embedding(vocab_size,50,input_length=seq_len, dropout=0.2, weights=[emb], trainable = False),
        Dropout(0.25),
        Convolution1D(64,5,border_mode='same',activation='relu'),
        Dropout(0.25),
        MaxPooling1D(),
        Flatten(),
        Dense(100,activation='relu'),
        Dropout(0.7),
        Dense(1,activation='sigmoid')
    ])

In [62]:
model.compile(loss = 'binary_crossentropy',optimizer=Adam(), metrics=['accuracy'])

In [64]:
model.fit(trn,labels_train, validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 175s - loss: 0.5887 - acc: 0.6808 - val_loss: 0.4864 - val_acc: 0.7972
Epoch 2/2
12096/25000 [=============>................] - ETA: 55s - loss: 0.4964 - acc: 0.7660

KeyboardInterrupt: 

Fine tune the embedding layers as well

In [ ]:
model.layers[0].trainable = True
model.optimizer.lr = le-4
model.compile(loss = 'binary_crossentropy',optimizer=Adam(), metrics=['accuracy'])
model.fit(trn,labels_train, validation_data=(test,labels_test),nb_epoch=1,batch_size=64)

In [ ]:
model.save_weights(model_path+'glove50.h5')